In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import csv
import os
from os import listdir


# notes
- aim for a shared phi
- different classifiers for each environment, w_e depends on e
2 phases, sort of:
1. minimize the sum across all environments, the expected loss with env-specific weight vectors
2. fix the phi from step 1. learn a w which is shared among the environments

Why do we want that?
- from the DARE paper (Rosenfeld '22): sharing phi for environments is okay. 
    - even if you "cheat" when you create the featurizer by looking at test env, it doesn't improve the perf. that much
- the bottleneck is actually in learning a _robust w_ .

1. `learn_featurizer` = small neural network, calculate loss carefully
2. `learn_classifier` = calculate average loss

## datasets
- Waterbirds, from Sagawa '22
    - https://github.com/kohpangwei/group_DRO
    - DARE, ISR used


How does it work
- put in a NN with whatever layers, and a last one
- first train neural network
- loss is sum with different w's on different environments



In [3]:
# celebA
# H, W = 218, 178

#waterbirds:
H, W = 300, 500

In [7]:
# get the path/directory
albatross_dir = "./waterbirds/001.Black_footed_Albatross"
blackbird_dir = "./waterbirds/009.Brewer_Blackbird"
abirds = []
bbirds = []
for img in os.listdir(albatross_dir):
	if (img.endswith(".jpg")):
		abirds.append(img)
for img in os.listdir(blackbird_dir):
	if (img.endswith(".jpg")):
		bbirds.append(img)

# get waterbirds; returns dict birds
def get_place():
	'''
	returns:    birds, dict with keys 
					'0' for land
					'1' for water
				maps to an iterable of filenames
	'''
	# birds = {'0':[], '1':[]}
	place = {}
	with open('./waterbirds/metadata.csv') as f:
		reader = csv.DictReader(f)
		for row in reader:
			place[row['img_filename']] = place[row['place']]
			# birds[row['place']].append(row['img_filename'])
	return place


def get_two_birds():
	birds = {'0':[], '1':[] } # 0 is land, 1 is water (for background)
	
    

In [ ]:
class WaterBirdsDataset(Dataset):
    def __init__(self, data_f, dev, transform=None):
        self.transform = transform
        self._load_data(data_f, dev)

    def _load_data(self, data_f, dev):
        data1 = get_two_birds()
        for idx, k in enumerate(data1):
            tmp = torch.Tensor(k)
            tmp = tmp.view(1, H, W)
            if self.transform:
                tmp = self.transform(tmp)
            self.d[idx, :, :, :] = tmp

        self.d = self.d.to(dev)

    def __len__(self):
        return self.d.size()[0]

    def __getitem__(self, idx):
        return self.d[idx, :, :]

In [ ]:
batch_size = 4

# first download
# download(BASE_URL + f_name, f_name)

transforms = torch.nn.Sequential(
    transforms.CenterCrop(10),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
)

print("Processing dataset ...")
train_data = WaterBirdsDataset(
    transform=transforms.Compose([
        transforms.CenterCrop(H,W)
        transforms.Normalize((0.0,), (255.0,))
        ]),
)
print(f"... done. Total {len(train_data)} data entries.")

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    drop_last=True,
)

In [11]:
class BasicNet(nn.Module):
    def __init__(self):
        super(BasicNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        '''
            Computes the network's forward pass on the input tensor.
            Does not apply a softmax or other activation functions.

            Arguments:
                self: This object.
                x: The tensor to compute the forward pass on.
        '''
        x = self.fc1(x)
        return x

In [22]:
# phase 1 training loop

def fit(net, X, y, n_epochs=201):
    '''
    Trains the neural network with CrossEntropyLoss and an Adam optimizer on
    the training set X with training labels Y for n_epochs epochs.

    Arguments:
        net: The neural network to train
        X: n x d tensor
        y: n x 1 tensor
        n_epochs: The number of epochs to train with batch gradient descent.

    Returns:
        List of losses at every epoch, including before training
        (for use in plot_cafe_loss).
    '''
    # stepsize = 0.1
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    losses = []

    for i in range(n_epochs):
        optimizer.zero_grad() 
        loss = loss_fn(net(X),y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    return losses
    
net = BasicNet()
birds = get_waterbirds()

for e in envs:  
    print(X.shape)
    losses = fit(net, X, y)
    print(losses[0], "first")
    print(losses[-1], "last")

TypeError: get_waterbirds() takes 0 positional arguments but 1 was given

In [ ]:
# phase 2 training loop